<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/notebooks/2021_0510ccap_mini_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CCAP デモ
- date: 2021_0510
- filename: 2021_0510ccap_demo.pynb
- author: 浅川伸一
- 概要:

In [ ]:
#colab 上で MeCab をインストール際に，次行をコメントアウト
!pip install mecab-python3==0.996.3
import MeCab

!pip install jaconv
import jaconv

import matplotlib.pyplot as plot
!pip install japanize_matplotlib
import japanize_matplotlib 

In [ ]:
from google.colab import files
uploaded = files.upload()  # ここで tsne.py, ja_util.py, linalg_util.py, 2021_0510.pickle を 


In [23]:
#!ls -l *.pickle

In [ ]:
import pickle
D = pickle.load(open("2021_0510.pickle", "rb"))

print(list(D))

In [ ]:
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz

In [33]:
import numpy as np
# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

# word2vec データ処理のため gensim を使う
import os
import sys
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

print('# word2vec データの読み込み')
print('# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...', end="")
# ファイルの所在に応じて変更してください
w2v_base = '.'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_base = '/Users/asakawa/study/2016wikipedia/'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 

w2v = asakawa_w2v
print('done')

# word2vec データの読み込み
# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...done


In [ ]:
%load_ext autoreload
%autoreload 2
import linalg_util
import tsne

In [34]:
%%time
M_animal_norm = np.array([x/np.linalg.norm(x) for x in D['M_animal']])
R_animal = M_animal_norm.dot(M_animal_norm.T)
print(f'R_animal.shape: {R_animal.shape}')

M_onmtp_norm = np.array([x/np.linalg.norm(x) for x in D['M_onmtp']])
R_onmtp = M_onmtp_norm.dot(M_onmtp_norm.T)
print(f'R_onmtp.shape: {R_onmtp.shape}')

M_ntt_norm = np.array([x/np.linalg.norm(x) for x in D['M_ntt']]) 
R_ntt = M_ntt_norm.dot(M_ntt_norm.T)
print(f'R_ntt.shape: {R_ntt.shape}')

R_animal.shape: (647, 647)
R_onmtp.shape: (1636, 1636)
R_ntt.shape: (4000, 4000)
CPU times: user 611 ms, sys: 413 ms, total: 1.02 s
Wall time: 783 ms


In [35]:
#基本単語を 元にした オノマトペ単語の近隣語
def sim_O(word, n_best=10, 
          X_norm=M_onmtp_norm, Y_norm=M_ntt_norm,
          X_list=D['onmtp_names'], Y_list=D['ntt3k_names']):
    
    SimMat = X_norm.dot(Y_norm.T)          # X x Yの類似度行列
    assert word in Y_list, 'not exit'
    
    idx = Y_list.index(word)
    sim_vec = SimMat.T[idx]
    
    idx_list = np.argsort(sim_vec)[::-1]
    return [[X_list[idx],sim_vec[idx]] for idx in idx_list]


#オノマトペを元にした 基本単語の近隣語
def sim_B(word, n_best=10, 
          X_norm=M_ntt_norm, Y_norm=M_onmtp_norm, 
          X_list=D['ntt3k_names'], Y_list=D['onmtp_names']):
    
    SimMat = X_norm.dot(Y_norm.T)       # X x Y の類似度行列
    assert word in Y_list, 'not exit'
    
    idx = Y_list.index(word)
    sim_vec = SimMat[idx]
    
    idx_list = np.argsort(sim_vec)[::-1]
    return [[X_list[idx], sim_vec[idx]] for idx in idx_list]

In [38]:
n = 5
topn = 10

for i, w in enumerate(np.random.choice(D['onmtp_names'], n)):
    print(f'{i+1:3d} {w:<10s}', end=":")
    for x, p in sim_B(w)[:topn]:
        print(f'({x}:{p:.3f}) ', end=" ")
    print()

print()
for i, w in enumerate(np.random.choice(D['ntt3k_names'], n)):
    print(f'{i+1:3d} {w:<10s}', end=":")
    for x, p in sim_O(w)[:topn]:
        print(f'({x}:{p:.3f}) ', end=" ")
    print()

  1 ギンギン      :(成果:0.374)  (場合:0.335)  (権利:0.326)  (再び:0.320)  (手続き:0.314)  (安定:0.310)  (要望:0.299)  (外国:0.288)  (融資:0.286)  (取材:0.283)  
  2 すかっ       :(隊:0.366)  (異なる:0.311)  (似る:0.292)  (印象:0.289)  (十六:0.285)  (中央:0.284)  (大使館:0.279)  (講演:0.269)  (推薦:0.262)  (続ける:0.256)  
  3 コツコツ      :(値上げ:0.265)  (安定:0.265)  (高:0.262)  (両:0.262)  (伝える:0.231)  (際:0.229)  (赤字:0.223)  (動き:0.214)  (交換:0.209)  (始める:0.208)  
  4 がぶっ       :(演出:0.298)  (年度:0.212)  (改めて:0.203)  (大蔵省:0.178)  (需要:0.169)  (市長:0.149)  (選挙:0.135)  (抑える:0.134)  (改革:0.132)  (重視:0.123)  
  5 ぱくり       :(市長:0.225)  (支払う:0.204)  (地震:0.193)  (不良:0.190)  (運用:0.183)  (夜:0.172)  (開放:0.172)  (及ぶ:0.169)  (専門:0.167)  (解消:0.157)  

  1 少し        :(だんだん:0.489)  (どんどん:0.455)  (しっかり:0.411)  (ふらふら:0.369)  (ボロボロ:0.354)  (きっ:0.341)  (バラバラ:0.335)  (ぼろぼろ:0.318)  (フラフラ:0.314)  (ぴったり:0.311)  
  2 距離        :(ジグザグ:0.344)  (ぎりぎり:0.333)  (トン:0.306)  (ジリジリ:0.267)  (ドット:0.264)  (ギリギリ:0.246)  (パー:0.238)  (スルスル:0.238)  (ボタン:0.218)  (ズルズル:0.204)  
  3 賛成   

In [53]:
topn = 5
for word in ['大学', '学校']:
    print(word, end=":  ")
    for x, p in sim_O(word)[:topn]:
        print(f'({x}:{p:.3f}) ', end=" ")
    print()

大学:  (バキン:0.227)  (ボン:0.218)  (クラリ:0.210)  (ワラワラ:0.196)  (ハラリ:0.175)  
学校:  (のびのび:0.254)  (バキン:0.241)  (あくせく:0.234)  (がっ:0.221)  (ハキハキ:0.207)  


In [54]:
topn = 5
for word in ['ガッカリ', 'プリプリ']:
    print(word, end=":  ")
    for x, p in sim_B(word)[:topn]:
        print(f'({x}:{p:.3f}) ', end=" ")
    print()

ガッカリ:  (ドル:0.291)  (大使:0.288)  (届く:0.277)  (心:0.254)  (印象:0.242)  
プリプリ:  (形:0.419)  (対抗:0.340)  (予算:0.316)  (税:0.293)  (実績:0.282)  
